In [1]:
import torch
import torch.nn as nn
from model import Transformer 
from config import get_config, get_weights_file_path
from train import get_model, get_ds, greedy_decode
import altair as alt 
import pandas as pd
import numpy as np 
import warnings
warnings.filterwarnings("ignore")

In [2]:
# define the device
device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # check if GPU parallel processing is aviliable
print(f"Using device: {device}")

Using device: cpu
